Leer Texto de imagenes

In [9]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import csv
import matplotlib.pyplot as plt
import cv2
import os

In [10]:
# Obtener la lista de archivos en la carpeta
folder_path = 'data/input/imagenes_prueba/'
image_files = os.listdir(folder_path)

# Cargar el procesador y el modelo
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

# Iterar sobre cada imagen en la carpeta
for image_file in image_files:
    if image_file.endswith('.png') or image_file.endswith('.jpg') or image_file.endswith('.jpeg'):
        # Cargar la imagen
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path)

        # Preprocesamiento de la imagen
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

        # Procesar la imagen y generar el texto
        pixel_values = processor(images=binary_image, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Imprimir el texto generado
        print("Texto reconocido en", image_file, ":", generated_text)

        # Escribir el texto reconocido en un archivo CSV
        output_csv = 'data/output/output_text/output_text_trOCR.csv'
        with open(output_csv, 'a', newline='', encoding='utf-8') as csvfile:  # Use 'a' for append mode
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow([image_file, generated_text])

print("Texto reconocido de todas las imágenes guardado en", output_csv)

e:\ANACONDA\envs\SATURDAYSAI-TENSOR-CV2\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Invalid image shape. Expected either 4 or 3 dimensions, but got 2 dimensions.